### DualSPH / test-16

- Reference test case: https://www.spheric-sph.org/tests/test-16
- Code changes in branch [spheric_test16](https://github.com/jgphpc/DualSPHysics/blob/spheric_test16/)
    - src/source/JSph.cpp:[JSph::LoadCodeParticles](https://github.com/jgphpc/DualSPHysics/blob/spheric_test16/src/source/JSph.cpp#L1260)    

#### GenCase

- Create the initial inputs files with: `GenCase.sh eff.033_Def eff.033_out/eff.033 -save:+all`
- Results:
  - [Gencase.out](https://github.com/jgphpc/DualSPHysics/blob/spheric_test16/examples/others/spheric/test16/cubeside_30/data/Gencase.out)
  - [30.xml](https://github.com/jgphpc/DualSPHysics/blob/spheric_test16/examples/others/spheric/test16/cubeside_30/30.xml)
  - [30.bi4(.gz)](https://github.com/jgphpc/DualSPHysics/blob/spheric_test16/examples/others/spheric/test16/cubeside_30/30.bi4.gz)

#### DualSPH

- Recompile with the changes from the [spheric_test16](https://github.com/jgphpc/DualSPHysics/blob/spheric_test16/) branch
- Launch with: `DualSPHysics5.2CPU_linux64 eff.033/eff.033_out/eff.033 eff.033_out -dirdataout data -svres -sv:csv`
- Results:
  - [Run.out](https://github.com/jgphpc/DualSPHysics/blob/spheric_test16/examples/others/spheric/test16/cubeside_30/data/Run.out)
  - [PartCsv_0000.csv(.gz)](https://github.com/jgphpc/DualSPHysics/blob/spheric_test16/examples/others/spheric/test16/cubeside_30/data/PartCsv_0000.csv.gz), etc...
  - *.bi4

In [20]:
# python3 -m ipykernel install --user --name=myvenv
import math as m
import numpy as np

In [4]:
# define CONSTANTS
p2 = m.pi / 2.  # 1.5707963267948966
p3 = m.pi / 3.  # 1.0471975511965976
k = 2 * m.pi
A = 4.*m.sqrt(2) / (3.*m.sqrt(3))

In [33]:
infile = '/tmp/0/sph/PartCsv_0000.csv'
data = np.genfromtxt(infile, delimiter=';', skip_header=1)

In [34]:
data[:, 2][0:5]

array([0.   , 0.033, 0.066, 0.099, 0.132])

In [35]:
pos_x = data[:, 0]  # numpy.ndarray
pos_y = data[:, 1]
pos_z = data[:, 2]
#
vel_x = data[:, 4]
vel_y = data[:, 5]
vel_z = data[:, 6]

- np = $30^3 = 27000$
- ( np = $32^3 = 32769$ )

In [36]:
# len(vel_x) == 32**3
len(vel_x) == 30**3

True

# Average kinetic energy per unit of mass from $(v_x, v_y, v_z)_{t0}$
$\epsilon$ = U0**2 / 2 at t=0

In [37]:
print(f"{min(vel_x):.12f}, {min(vel_y):.12f}, {min(vel_z):.12f}" )
print(f"{max(vel_x)}, {max(vel_y)}, {max(vel_z)}" )

-1.409248000000, -1.409248000000, -1.409248000000
1.4109951, 1.4109951, 1.4109951


In [38]:
# average kinetic energy per unit of mass at t=0 should be U0**2 / 2
ss_a = 0.0
for ii in range(len(vel_x)):
    ss_a += vel_x[ii]*vel_x[ii] + vel_y[ii]*vel_y[ii] + vel_z[ii]*vel_z[ii]

ss_a/2

13324.098369180552

In [39]:
# average kinetic energy per unit of mass at t=0 should be U0**2 / 2
ss_b = sum(vx**2 + vy**2 + vz**2 for vx, vy, vz in zip(vel_x, vel_y, vel_z))
ss_b == ss_a

True

# Average kinetic energy per unit of mass from $(x, y, z)_{t0}$
$\epsilon$ = U0**2 / 2 at t=0

In [40]:
print(f"{min(pos_x):.12f}, {min(pos_y):.12f}, {min(pos_z):.12f}" )
print(f"{max(pos_x)}, {max(pos_y)}, {max(pos_z)}" )

0.000000000000, 0.000000000000, 0.000000000000
0.95700002, 0.95700002, 0.95700002


In [41]:
vel_x_from_pos = []
vel_y_from_pos = []
vel_z_from_pos = []
for ii in range(len(pos_x)):
    kx = k * pos_x[ii]
    ky = k * pos_y[ii]
    kz = k * pos_z[ii]
    #
    vel_x_from_pos.append(
        A * (
            m.sin(kx-p3) * m.cos(ky+p3) * m.sin(kz+p2) -
            m.cos(kz-p3) * m.sin(kx+p3) * m.sin(ky+p2)
        )
    )
    #
    vel_y_from_pos.append(
        A * (
            m.sin(ky-p3) * m.cos(kz+p3) * m.sin(kx+p2) -
            m.cos(kx-p3) * m.sin(ky+p3) * m.sin(kz+p2)
        )
    )
    #
    vel_z_from_pos.append(
        A * (
            m.sin(kz-p3) * m.cos(kx+p3) * m.sin(ky+p2) -
            m.cos(ky-p3) * m.sin(kz+p3) * m.sin(kx+p2)
        )
    )    

In [42]:
ss_c = sum(vx**2 + vy**2 + vz**2 for vx, vy, vz in zip(vel_x_from_pos, vel_y_from_pos, vel_z_from_pos))
ss_c/2

13324.096777900571

In [43]:
ss_a - ss_c < 0.005

True

# Round off

In [18]:
vel_x_from_pos == vel_x

array([False, False, False, ..., False, False, False])

In [24]:
print(vel_x_from_pos[0:3])
print(vel_x[0:3])
#
[ round(xx, 6) for xx in vel_x_from_pos[0:3] ]

[-0.9428090415820636, -1.0907010407190776, -1.1918693799691533]
[-0.94280905 -1.0907011  -1.1918694 ]


[-0.942809, -1.090701, -1.191869]